# Data Cleaning with Apache Spark

- Data cleaning is preparing raw data for use in processing pipelines
- Data cleaning is a necessary part of any production data system. If your data isn't "clean", it's not trustworthy and could cause problems later on.
- There are many tasks that could fall under the data cleaning umbrella. A few of these include reformatting or replacing text; performing calculations based on the data; and removing garbage or incomplete data.
- Most data cleaning systems have two big problems: optimizing performance and organizing the flow of data.
-  Spark lets you scale your data processing capacity as your requirements evolve. Beyond the performance issues, dealing with large quantities of data requires a process, or pipeline of steps. Spark allows management of many complex tasks within a single framework.

## Spark Schemas
- A primary function of data cleaning is to verify all data is in the expected format. Spark provides a built-in ability to validate datasets with schemas.
-  You may have used schemas before with databases or XML; Spark is similar. A schema defines and validates the number and types of columns for a given DataFrame. A schema can contain many different types of fields - integers, floats, dates, strings, and even arrays or mapping structures.
-  A defined schema allows Spark to `filter` out data that doesn't conform during read, `ensuring expected correctness`. In addition, schemas also have `performance benefits`.
-  Normally a data import will try to infer a schema on read - this requires reading the data twice. Defining a schema limits this to a single read operation.

In [ ]:
# import pyspark.sql.types
from pyspark.sql.types import *

peopleSchema = StructType([
    # Define the name field
    StructField('name', StringType(), True),
    # Add the age field
    StructField('age', IntegerType(), True),
    # Add the city field
    StructField('city', StringType(), True)
 ])

# Read CSV containing data
people_df = spark.read.format('csv').load(name='rawdata.csv', schema=peopleSchema)


## Immutability and lazy processing

- Python variables are fully mutable. The values can be changed at any given time, assuming the scope of the variable is valid.
- While very flexible, this does present problems anytime there are multiple concurrent components trying to modify the same data. Most languages work around these issues using constructs like mutexes, semaphores, etc. This can add complexity, especially with non-trivial programs.
- Spark Data Frames are immutable,  this means Spark Data Frames are defined once and are not modifiable after initialization (Unable to be directly modified). Immutability is a concept of functional programming and Spark is designed ot use immutable objects.
- If the variable name is reused, the original data is removed (assuming it's not in use elsewhere) and the variable name is reassigned to the new data.(Re-created if reassigned)
-  While this seems inefficient, it actually allows Spark to share data between all cluster components. It can do so without worry about concurrent data objects.(Able to be shared efficiently)

In [ ]:
# Define a new dataframe:
voter_df = spark.read.csv('voterdata.csv')

# Making changes: This does not modfiy the original data frame but it actually copeis the orignal definition
# Adds/Applies the transformation and assigns it to the variable.
voter_df = voter_df.withColumn('fullyear',voter_df.year + 2000)

# Drop the column year
voter_df = voter_df.drop(voter_df.year)

# Action processes all transformations
voter_df.count()



## Lazy Processing
- Very little happens until an action is performed
- When Spark works with big data the trick that it uses is that no data was actually read / added / modified, only thing gets updated are the instructions (aka, Transformations) for what we wanted Spark to do.This functionality allows Spark to perform the most efficient set of operations to get the desired result
- Lazy processing operations will usually return in about the same amount of time regardless of the actual quantity of data. Remember that this is due to Spark not performing any transformations until an action is requested.


In [ ]:
# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load('AA_DFW_2018.csv.gz')

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show()

## Understanding Parquet

 - No defined schema - there are no data types included, nor column names (beyond a header row)
 - Nested data requires special handling - Using content containing a comma (or another delimiter) requires escaping. Using the escape character within content requires even further escaping
 -  Encoding format limited - The available encoding formats are limited depending on the language used.

 - Spark has some specific problems processing CSV data.
    -  CSV files are quite slow to import and parse.
    -  The files cannot be shared between workers during the import process.
    -  If no schema is defined, all data must be read before a schema can be inferred.
    - Spark has feature known as predicate pushdown. Basically, this is the idea of ordering tasks to do the least amount of work. Filtering data prior to processing is one of the primary optimizations of predicate pushdown. This drastically reduces the amount of information that must be processed in large data sets. Unfortunately, you cannot filter the CSV data via `predicate pushdown`
    - Finally, Spark processes are often multi-step and may utilize an intermediate file representation. These representations allow data to be used later without regenerating the data from source. Using CSV would instead require a significant amount of extra work defining schemas, encoding formats, etc.

- `Parquet` is a compressed columnar data format developed for use in any Hadoop based system. This includes Spark, Hadoop, Apache Impala, and so forth.
- The Parquet format is structured with data accessible in chunks, allowing efficient read / write operations without processing the entire file. This structured format supports Spark's predicate pushdown functionality, providing significant performance improvement.
- Parquet files automatically include schema information and handle data encoding. This is perfect for intermediary or on-disk representation of processed data. Note that Parquet files are a binary file format and can only be used with the proper tools. This is in contrast to CSV files which can be edited with any text editor.

`Note`
The Parquet format is a columnar data store, allowing Spark to use predicate pushdown. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets.

In [ ]:
# Reading Parquet files
# The long-form versions of each permit extra option flags, such as when overwriting an existing parquet file.
df = spark.read.format('parquet').load('filename.parquet')
df = spark.read.parquet('filename.parquet')

# Writing Parquet files
df.write.format('parquet').save('filename.parquet')
df.write.parquet('filename.parquet')

# Parquet perfect for performing SQL operations or using it as backing stores for SparkSQL operations 
# we get all the performance benefits primarily defined schemas and the available use of predicate pushdown
flight_df = spark.read.parquet('flights.parquet')
flight_df.createOrReplaceTempView('flights')
short_flights_df = spark.sql('SELECT * FROM flights WHERE flightduration < 100')

In [ ]:

# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

In [ ]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet('AA_DFW_ALL.parquet')

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

## Spark Column Operations

- DataFrames are made up of rows & columns and are generally analogous to a database table.
- DataFrames are immutable: any change to the structure or content of the data creates a new DataFrame.
- DataFrames are modified through the use of transformations.
- Negate with `~`
- Filtering includes only rows that satisfy the requirements defined in the argument.

In [ ]:
# Return rows where name starts with "M" 
voter_df.filter(voter_df.name.like('M%'))

# Return name and position only
voters = voter_df.select('name', 'position')

# Filter/Where
voter_df.filter(voter_df.date > '1/1/2019') # or voter_df.where(...)

# Select
voter_df.select(voter_df.name)

# withColumn
voter_df.withColumn('year', voter_df.date.year)

# drop Column
voter_df.drop('unused_column')

# Remove/Keep nulls
voter_df.filter(voter_df['name'].isNotNull())
voter_df.where(~ voter_df._c1.isNull())

# Remove Odd Entries
voter_df.filter(voter_df.date.year > 1800)
voter_df.where(voter_df['_c0'].contains('VOTE'))


In [ ]:
# Column String Transformations
import pyspark.sql.functions as F

# Applied per column as transformation
voter_df.withColumn('upper', F.upper('name'))

# Can create intermediary columns 
voter_df.withColumn('splits', F.split('name', ' '))

# Can cast to other types
voter_df.withColumn('year', voter_df['_c4'].cast(IntegerType()))

## Array Type Column Functions

- While performing data cleaning with Spark, you may need to interact with `ArrayType() `columns. These are analogous to lists in normal python environments. 
- Functions we will use are
   1. `.size()`, which returns the number of items present in the specified ArrayType() argument. 
   2. `.getItem()`. It takes an index argument and returns the item present at that index in the list column.

In [ ]:
# Show the distinct VOTER_NAME entries
voter_df.select('VOTER_NAME').distinct().show(40, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains('_'))

# Show the distinct VOTER_NAME entries again
voter_df.select('VOTER_NAME').distinct().show(40, truncate=False)

# Select Name & State column for ID greater than 3000
users_df.filter('ID > 3000').select("Name", "State")

In [ ]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))

# Drop the splits column
voter_df = voter_df.drop('splits')

# Show the voter_df DataFrame
voter_df.show()

## Conditional Operations in a Dataframe

The DataFrame transformations we've covered thus far are blanket transformations, meaning they're applied regardless of the data.

When you want to conditionally change some aspect of the contents. Spark provides some built in conditional clauses which act similar to an if / then / else statement in a traditional programming environment. While it is possible to perform a traditional if / then / else style statement in Spark, it can lead to serious performance degradation as each row of a DataFrame would be evaluated independently. Using the optimized, built-in conditionals alleviates this. There are two components to the conditional clauses: `.when()`, and the optional `.otherwise()`.

The .when() clause is a method available from the pyspark.sql.functions library

Syntax : `when(<if condition>, <then x>)`



In [ ]:
# Ex. 1
df.select(df.Name, df.Age, F.when(df.Age >= 18, "Adult"))

# Ex. 2
df.select(df.Name, df.Age,           
          .when(df.Age >= 18, "Adult")
          .when(df.Age < 18, "Minor"))

# Ex. 3
df.select(df.Name, df.Age,          
          .when(df.Age >= 18, "Adult") 
          .otherwise("Minor"))

# Add a column to voter_df for any voter with the title **Councilmember**
voter_df = voter_df.withColumn('random_val',
                               when(voter_df.TITLE == 'Councilmember', F.rand()))

# Show some of the DataFrame rows, noting whether the when clause worked
voter_df.show()

# Add a column to voter_df for a voter based on their position
voter_df = voter_df.withColumn('random_val',
                               when(voter_df.TITLE == 'Councilmember', F.rand())
                               .when(voter_df.TITLE == 'Mayor', 2)
                               .otherwise(0))

# Show some of the DataFrame rows
voter_df.show()

# Use the .filter() clause with random_val
voter_df.filter(voter_df.random_val==0).show()

## User Defined Functions

- A user defined function, or UDF, is a Python method that the user writes to perform a specific bit of logic.
- Once written, the method is called via the pyspark.sql.functions.udf() method. The result is stored as a variable and can be called as a normal Spark function.

In [ ]:
# Define a Python method 
def reverseString(mystr):
    return mystr[::-1]
    
# Wrap the function and store as a variable
# Method takes two arguments one is the function name and second is the data type of the return object
# This can be any of the options in pyspark.sql.types, and can even be a more complex type, including a fully defined schema object.
# Most often, you'll return either a simple object type, or perhaps an ArrayType
udfReverseString = udf(reverseString, StringType())

# Use with Spark
user_df = user_df.withColumn('ReverseName', udfReverseString(user_df.Name))

# Argument less example
def sortingCap():
    return random.choice(['G', 'H', 'R', 'S'])

udfSortingCap = udf(sortingCap, StringType())

user_df = user_df.withColumn('Class', udfSortingCap())


def getFirstAndMiddle(names):
  # Return a space separated string of names
  return ' '.join(names[:-1])

# Define the method as a UDF
udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())

# Create a new column using your UDF
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))

# Show the DataFrame
voter_df.show()


## Partitioning and lazy processing

- `Spark breaks DataFrames into partitions, or chunks of data.` These partitions can be automatically defined, enlarged, shrunk, and can differ greatly based on the type of Spark cluster being used. 

- The size of the partition does vary, but generally try to keep your partition sizes equal.Each partition is handled independently. This is part of what provides the performance levels and horizontal scaling ability in Spark. If a Spark node doesn't need to compete for resources, nor consult with other Spark nodes for answers, it can reliably schedule the processing for the best performance.

- In Spark, any transformation operation is `lazy`; it's more like a recipe than a command. `It defines what should be done to a DataFrame rather than actually doing it`.

- Most operations in Spark are actually `transformations`, including .withColumn(), .select(), .filter(), and so forth. The set of transformations you define are only executed when you run a Spark `action`.This includes .count(), .write(), etc - anything that requires the transformations to be run to properly obtain an answer. 

-  Spark can reorder transformations for the best performance. Usually this isn't noticeable, but can occasionally cause unexpected behavior, such as IDs not being added until after other transformations have completed. This doesn't actually cause a problem but the data can look unusual if you don't know what to expect.

## Adding IDs

- Relational databases tend to have a field used to identify the row, whether it is for an actual relationship reference, or just for data identification. These IDs are typically an integer that increases in value, is sequential, and most importantly unique.
-  The problem with these IDs is they're not very parallel in nature. Given that the values are given out sequentially, if there are multiple workers, they must all refer to a common source for the next entry. This is OK in a single server environment, but in a distributed platform such as Spark, it creates some undue bottlenecks.
- Spark has a built-in function called `monotonically_increasing_id()`, designed to provide an integer ID that increases in value and is unique. These IDs are not necessarily sequential - there can be gaps, often quite large, between values.
- Unlike a normal relational ID, Spark's is completely parallel - each partition is allocated up to 8 billion IDs that can be assigned. Notice that the ID fields in the sample table are integers, increasing in value, but are not sequential.
-  It's a little out scope, but the IDs are a 64-bit number effectively split into groups based on the Spark partition. Each group contains 8.4 billion IDs, and there are 2.1 billion possible groups, none of which overlap.


In [ ]:
# Select all the unique council voters
voter_df = df.select(df["VOTER NAME"]).distinct()

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

# Add a ROW_ID
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)

In [ ]:
# Print the number of partitions in each DataFrame
print("\nThere are %d partitions in the voter_df DataFrame.\n" % voter_df.rdd.getNumPartitions())
print("\nThere are %d partitions in the voter_df_single DataFrame.\n" % voter_df_single.rdd.getNumPartitions())

# Add a ROW_ID field to each DataFrame
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())
voter_df_single = voter_df_single.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the top 10 IDs in each DataFrame 
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)
voter_df_single.orderBy(voter_df_single.ROW_ID.desc()).show(10)

In [ ]:
# Determine the highest ROW_ID and save it in previous_max_ID
previous_max_ID = voter_df_march.select('ROW_ID').rdd.max()[0]

# Add a ROW_ID column to voter_df_april starting at the desired value
voter_df_april = voter_df_april.withColumn('ROW_ID', previous_max_ID + F.monotonically_increasing_id())

# Show the ROW_ID from both DataFrames and compare
voter_df_march.select('ROW_ID').show()
voter_df_april.select('ROW_ID').show()

# Caching

- Caching in Spark refers to storing the results of a DataFrame in memory or on disk of the processing nodes in a cluster.
- Caching improves the speed for subsequent transformations or actions as the data likely no longer needs to be retrieved from the original data source.
- Using caching reduces the resource utilization of the cluster - there is less need to access the storage, networking, and CPU of the Spark nodes as the data is likely already present.

## Disadvantages of caching
- Very large data sets may not fit in the memory reserved for cached DataFrames.
- Depending on the later transformations requested, the cache may not do anything to help performance. 
- If a data set does not stay cached in memory, it may be persisted to disk 
- Depending on the disk configuration of a Spark cluster, this may not be a large performance improvement
- If you're reading from a local network resource and have slow local disk I/O, it may be better to avoid caching the objects.
- Finally, the lifetime of a cached object is not guaranteed. Spark handles regenerating DataFrames for you automatically, but this can cause delays in processing.

## Caching tips
- Caching is incredibly useful, but only if you plan to use the DataFrame again. If you only need it for a single task, it's not worth caching. The best way to gauge performance with caching is to test various configurations. Try caching your DataFrames at various points in the processing cycle and check if it improves your processing time.
- Try to cache in memory or fast NVMe / SSD storage. While still slower than main memory modern SSD based storage is drastically faster than spinning disk. Local spinning hard drives can still be useful if you are processing large DataFrames that require a lot of steps to generate, or must be accessed over the Internet.
- Testing this is crucial. If normal caching doesn't seem to work, try creating intermediate Parquet representations. These can provide a checkpoint in case a job fails mid-task and can still be used with caching to further improve performance.
- Finally, you can manually stop caching a DataFrame when you're finished with it. This frees up cache resources for other DataFrames.

## Implementing Caching
- Call `.cache()` on the DataFrame before Action 
- `.cache()` is a Spark transformation - nothing is actually cached until an action is called.


In [ ]:
voter_df = spark.read.csv('voter_data.txt.gz')
voter_df.cache().count()

voter_df = voter_df.withColumn('ID', monotonically_increasing_id())
voter_df = voter_df.cache()
voter_df.show()

# Check `.is_cached` to determine cache status 
print(voter_df.is_cached)

# Call.unpersist() when finished with DataFrame
voter_df.unpersist()

In [ ]:
# Consider why the first run takes longer even though you've told it to cache() the DataFrame.
# Remember that even though you've applied the caching transformation, it doesn't take effect until an action is run.
# The action instantiates the caching after the distinct() function completes.
# The second time, there is no need to recalculate anything so it returns almost immediately.

start_time = time.time()

# Add caching to the unique rows in departures_df
departures_df = departures_df.distinct().cache()

# Count the unique rows in departures_df, noting how long the operation takes
print("Counting %d rows took %f seconds" % (departures_df.count(), time.time() - start_time))

# Count the rows again, noting the variance in time of a cached DataFrame
start_time = time.time()
print("Counting %d rows again took %f seconds" % (departures_df.count(), time.time() - start_time))


In [ ]:
# Determine if departures_df is in the cache
print("Is departures_df cached?: %s" % departures_df.is_cached)
print("Removing departures_df from cache")

# Remove departures_df from the cache
departures_df.unpersist()

# Check the cache status again
print("Is departures_df cached?: %s" % departures_df.is_cached)

## Improve import performance

- Spark clusters consist of two types of processes - one driver process and as many worker processes as required. 
- The driver handles task assignments and consolidation of the data results from the workers. The workers typically handle the actual transformation / action tasks of a Spark job.
-  Once assigned tasks, they operate fairly independently and report results back to the driver. It is possible to have a single node Spark cluster but you'll rarely see this in a production environment. There are different ways to run Spark clusters - the method used depends on your specific environment.

### Import Performance

- When importing data to Spark DataFrames, it's important to understand how the cluster implements the job. The process varies depending on the type of task, but it's safe to assume that the more import objects available, the better the cluster can divvy up the job.
- This may not matter on a single node cluster, but with a larger cluster each worker can take part in the import process. In clearer terms, `one large file will perform considerably worse than many smaller ones`.

- Depending on the configuration of your cluster, you may not be able to process larger files, but could easily handle the same amount of data split between smaller files. Note you can define a single import statement, even if there are multiple files. You can use any form of standard wildcard symbol when defining the import filename. While less important, if objects are about the same size, the cluster will perform better than having a mix of very large and very small objects.

### Defined Schemas

Well-defined schemas in Spark drastically improve import performance. Without a schema defined, import tasks require reading the data multiple times to infer structure. This is very slow when you have a lot of data. Spark may not define the objects in the data the same as you would. Spark schemas also provide validation on import. This can save steps with data cleaning jobs and improve the overall processing time.

### Splitting for Performance

- There are various effective ways to split an object (files mostly) into more smaller objects. The first is to use built-in OS utilities such as split, cut, or awk. An example using split uses the -l argument with the number of lines to have per file (10000 in this case). The -d argument tells split to use numeric suffixes. The last two arguments are the name of the file to be split and the prefix to be used. Assuming 'largefile' has 10M records, we would have files named chunk-0000 through chunk-9999. 


- Another method is to use python (or any other language) to split the objects up as we see fit. Sometimes you may not have the tools available to split a large file. If you're going to be working with a DataFrame often, a simple method is to read in the single file then write it back out as parquet. We've done this in previous examples and it works well for later analysis even if the initial import is slow. It's important to note that if you're hitting limitations due to cluster sizing, try to do as little processing as possible before writing to parquet.

- Note that in certain circumstances the results may be reversed. This is a side effect of running as a single node cluster. Depending on the tasks required and resources available, it may occasionally take longer than expected. If you perform multiple runs of the tasks, you should see the full file import as generally slower than the split file import



In [ ]:
# Import the full and split files into DataFrames
full_df = spark.read.csv('departures_full.txt.gz')
split_df = spark.read.csv('departures_0*.txt.gz')

# Print the count and run time for each DataFrame
start_time_a = time.time()
print("Total rows in full DataFrame:\t%d" % full_df.count())
print("Time to run: %f" % (time.time() - start_time_a))

start_time_b = time.time()
print("Total rows in split DataFrame:\t%d" % split_df.count())
print("Time to run: %f" % (time.time() - start_time_b))

##  Configuration options

Spark has many available configuration settings controlling all aspects of the installation. These configurations can be modified to best match the specific needs for the cluster. The configurations are available in the configuration files, via the Spark web interface, and via the run-time code.

Reading configuration settings

`spark.conf.get(<configuration name>)`

Writing configuration settings

`spark.conf.set(<configuration name>)`

### Cluster Types

1.  Single node clusters, deploying all components on a single system (physical / VM / container).
2. Standalone clusters, with dedicated machines as the driver and workers. 
3. Managed clusters, meaning that the cluster components are handled by a third party cluster manager such as YARN, Mesos, or Kubernetes.

### Driver

- There is one driver per Spark cluster. The driver is responsible for several things, including the following: 
    1. Handling task assignment to the various nodes / processes in the cluster. 
    2. The driver monitors the state of all processes and tasks and handles any task retries. 
    3. The driver is also responsible for consolidating results from the other processes in the cluster. 
    4. The driver handles any access to shared data and verifies each worker process has the necessary resources (code, data, etc). 

- Doubling the memory compared to other nodes is recommended. This is useful for task monitoring and data consolidation tasks. 
- fast local storage is useful for running Spark in an ideal setup.

### Workers

- A Spark worker handles running tasks assigned by the driver and communicates those results back to the driver. Ideally, the worker has a copy of all code, data, and access to the necessary resources required to complete a given task. If any of these are unavailable, the worker must pause to obtain the resources. 

### Sizing a Cluster
- More worker nodes is often better than larger nodes. This can be especially obvious during import and export operations as there are more machines available to do the work.

- As with everything in Spark, test various configurations to find the correct balance for your workload. Assuming a cloud environment, 16 worker nodes may complete a job in an hour and cost $50 in resources. An 8 worker configuration might take 1.25 hrs but cost only half as much. 

- Finally, workers can make use of fast local storage (SSD / NVMe) for caching, intermediate files, etc.


In [ ]:
# Name of the Spark application instance
app_name = spark.conf.get('spark.app.name')

# Driver TCP port
driver_tcp_port = spark.conf.get('spark.driver.port')

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

In [ ]:
# Store the number of partitions in variable
before = departures_df.rdd.getNumPartitions()

# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 500)

# Recreate the DataFrame using the departures data file
departures_df = spark.read.csv('departures.txt.gz').distinct()

# Print the number of partitions for each instance
print("Partition count before change: %d" % before)
print("Partition count after change: %d" % departures_df.rdd.getNumPartitions())

## Spark Performance Impprovements

### Explain Plan

- To understand performance implications of Spark, you must be able to see what it's doing under the hood. The easiest way to do this is to use the .explain() function on a DataFrame.The result is the estimated plan that will be run to generate results from the DataFrame.

`voter_df = df.select(df['VOTER NAME']).distinct()`

`voter_df.explain()`

### Shuffling

- Spark distributes data amongst the various nodes in the cluster. A side effect of this is what is known as shuffling. Shuffling is the moving of data fragments to various workers as required to complete certain tasks. 
- Shuffling is useful and hides overall complexity from the user. That being said, it can be slow to complete the necessary transfers, especially if a few nodes require all the data.
- Shuffling lowers the overall throughput of the cluster as the workers must spend time waiting for the data to transfer. This limits the amount of available workers for the remaining tasks in the system. Shuffling is often a necessary component, but it's helpful to try to minimize it as much as possible.

### How to Limit Shuffling

- The DataFrame `.repartition()` function takes a single argument, the number of partitions requested. Repartitioning requires a full shuffle of data between nodes & processes and is quite costly. If you need to reduce the number of partitions, use the `.coalesce() `function instead. It takes a number of partitions smaller than the current one and consolidates the data without requiring a full data shuffle. Note: calling .coalesce() with a larger number of partitions does not actually do anything.

- The `.join()` function is a great use of Spark and provides a lot of power. Calling .join() indiscriminately can often cause shuffle operations, leading to increased cluster load & slower processing times. To avoid some of the shuffle operations when joining Spark DataFrames you can use the `.broadcast()` function.

- Broadcasting in Spark is a method to provide a copy of an object to each worker. When each worker has its own copy of the data, there is less need for communication between nodes. This limits data shuffles and it's more likely a node will fulfill tasks independently.

- Using broadcasting can drastically speed up .join() operations, especially if one of the DataFrames being joined is much smaller than the other. 

- To implement broadcasting, you must import the broadcast function from pyspark.sql.functions. Once imported, simply call the broadcast function with the name of the DataFrame you wish to broadcast.

- Note broadcasting can slow operations when using very small DataFrames or if you broadcast the larger DataFrame in a join. Spark will often optimize this for you, but as usual, run tests in your environment for best performance.

`from pyspark.sql.functions import broadcast`

`combined_df = df_1.join(broadcast(df_2))`





In [ ]:
# Join the flights_df and aiports_df DataFrames
normal_df = flights_df.join(airports_df, \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan
normal_df.explain()


# Import the broadcast method from pyspark.sql.functions
from pyspark.sql.functions import broadcast

# Join the flights_df and airports_df DataFrames using broadcasting
broadcast_df = flights_df.join(broadcast(airports_df), \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan and compare against the original
broadcast_df.explain()

start_time = time.time()
# Count the number of rows in the normal DataFrame
normal_count = normal_df.count()
normal_duration = time.time() - start_time

start_time = time.time()
# Count the number of rows in the broadcast DataFrame
broadcast_count = broadcast_df.count()
broadcast_duration = time.time() - start_time

# Print the counts and the duration of the tests
print("Normal count:\t\t%d\tduration: %f" % (normal_count, normal_duration))
print("Broadcast count:\t%d\tduration: %f" % (broadcast_count, broadcast_duration))

# Pipelines

- Data pipelines are simply the set of steps needed to move from an input data source, or sources, and convert it to the desired output. 

- A data pipeline can consist of any number of steps or components, and can span many systems, a full production data pipeline will likely communicate with many systems.

- A data pipeline typically consists of inputs, transformations, and the outputs of those steps. In addition, there is often validation and analysis steps before delivery of the data to the next user

- In Spark, a data pipeline is not a formally defined object, but rather a concept.Different than if you've used the Pipeline object in Spark.ML

What does a data pipeline look like ?

1. Input(s)
 - CSV, JSON, webservices, databases
2. Transformations 
 - withColumn(), .filter(), .drop()
3. Output(s)
 - CSV, Parquet, database
4. Validation (idea is to run some form of testing on the data to verify it is as expected.)
5. Analysis (often the final step before handing the data off to the next user. This can include things such as row counts, specific calculations, or pretty much anything that makes it easier for the user to consume the dataset.)



In [ ]:
# Import the data to a DataFrame
departures_df = spark.read.csv('2015-departures.csv.gz', header=True)

# Remove any duration of 0
departures_df = departures_df.filter(departures_df[3] > 0)

# Add an ID column
departures_df = departures_df.withColumn('id', F.monotonically_increasing_id())

# Write the file out to JSON format
departures_df.write.json('output.json', mode='overwrite')

# Order the dataframe with duration column
departures_df = departures_df.withColumn('Duration', departures_df['Duration'].cast(IntegerType()))

# Data handling techniques

- Spark's CSV parser can handle many common data issues via optional parameters. 
 - Blank lines are automatically removed (unless specifically instructed otherwise) when using the CSV parsing. 
- Comments can be removed with an optional named argument, `comment`, and specifying the character that any comment line would be defined by. Note that this handles lines that begin with a specific comment. Parsing more complex comment usage requires more involved procedures. 
- Header rows can be parsed via an optional parameter named `header`, and set to 'True' or 'False'. If no schema is defined, column names will be initially set as defined by the header. If a schema is defined, the row is not used as data, but the header names are otherwise ignored.

### Automatic column creation
When importing CSV data into Spark, it will automatically create DataFrame columns if it can. It will split a row of text from the CSV on a defined separator argument named `sep`. If sep is not defined, it will default to using a comma.
-  The CSV parser will still succeed in parsing data if the separator character is not within the string. It will store the entire row in a column named `_c0` by default. Using this trick allows parsing of nested or complex data.

In [ ]:
# Import the file to a DataFrame and perform a row count
annotations_df = spark.read.csv('annotations.csv.gz', sep='|')
full_count = annotations_df.count()

# Count the number of rows beginning with '#'
comment_count = annotations_df.where(col('_c0').startswith('#')).count()

# Import the file to a new DataFrame, without commented rows
no_comments_df = spark.read.csv('annotations.csv.gz', sep='|', comment='#')

# Count the new DataFrame and verify the difference is as expected
no_comments_count = no_comments_df.count()
print("Full count: %d\nComment count: %d\nRemaining count: %d" % (full_count, comment_count, full_count - comment_count))split


In [ ]:
# Split _c0 on the tab character and store the list in a variable
tmp_fields = F.split(annotations_df['_c0'], '\t')

# Create the colcount column on the DataFrame
annotations_df = annotations_df.withColumn('colcount', F.size(tmp_fields))

# Remove any rows containing fewer than 5 fields
annotations_df_filtered = annotations_df.filter(~ (annotations_df["colcount"] < 5))

# Count the number of rows
final_count = annotations_df_filtered.count()
print("Initial count: %d\nFinal count: %d" % (initial_count, final_count))

In [ ]:
# Split the content of _c0 on the tab character (aka, '\t')
split_cols = F.split(annotations_df['_c0'], '\t')

# Add the columns folder, filename, width, and height
split_df = annotations_df.withColumn('folder', split_cols.getItem(0))
split_df = split_df.withColumn('filename', split_cols.getItem(1))
split_df = split_df.withColumn('width', split_cols.getItem(2))
split_df = split_df.withColumn('height', split_cols.getItem(3))

# Add split_cols as a column
split_df = split_df.withColumn('split_cols', split_cols)

In [ ]:
def retriever(cols, colcount):
  # Return a list of dog data
  return cols[4:colcount]

# Define the method as a UDF
udfRetriever = F.udf(retriever, ArrayType(StringType()))

# Create a new column using your UDF
split_df = split_df.withColumn('dog_list', udfRetriever(split_df.split_cols, split_df.colcount))

# Remove the original column, split_cols, and the colcount
split_df = split_df.drop('_c0').drop('split_cols').drop('colcount')

## Data validation

- Validation is verifying that a dataset complies with an expected format. This can include verifying that the number of rows and columns is as expected. 
  - For example, is the row count within 2% of the previous month's row count? 
  - Another common test is do the data types match? If not specifically validated with a schema, does the content meet the requirements (only 9 characters or less, etc).
  -  Finally, you can validate against more complex rules. This includes verifying that the values of a set of sensor readings are within physically possible quantities.

- Validating via joins
 - One technique used to validate data in Spark is using joins to verify the content of a DataFrame matches a known set. Validating via a join will compare data against a set of known values. This could be a list of known ids, companies, addresses, etc. Joins make it easy to determine if data is present in a set. This could be only rows that are in one DataFrame, present in both, or present in neither.
 - Joins are also comparatively fast, especially vs validating individual rows against a long list of entries. The simplest example of this is using an inner join of two DataFrames to validate the data.

 `parsed_df = spark.read.parquet('parsed_data.parquet')`
 `company_df = spark.read.parquet('companies.parquet')`
 `verified_df = parsed_df.join(company_df, parsed_df.company == company_df.company)`

 - Complex rule validation is the idea of using Spark components to validate logic. This may be as simple as using the various Spark calculations to verify the number of columns in an irregular data set. The validation can also be applied against an external source: web service, local files, API calls. These rules are often implemented as a UDF to encapsulate the logic to one place and easily run against the content of a DataFrame. 

In [ ]:
# Rename the column in valid_folders_df
valid_folders_df = valid_folders_df.withColumnRenamed('_c0','folder')

# Count the number of rows in split_df
split_count = valid_folders_df.count()

# Join the DataFrames
joined_df = split_df.join(F.broadcast(valid_folders_df), "folder")

# Compare the number of rows remaining
joined_count = joined_df.count()
print("Before: %d\nAfter: %d" % (split_count, joined_count))

In [ ]:
# Determine the row counts for each DataFrame
split_count = split_df.count()
joined_count = joined_df.count()

# Create a DataFrame containing the invalid rows
invalid_df = split_df.join(F.broadcast(joined_df), 'folder', 'left_anti')

# Validate the count of the new DataFrame is as expected
invalid_count = invalid_df.count()
print(" split_df:\t%d\n joined_df:\t%d\n invalid_df: \t%d" % (split_count, joined_count, invalid_count))

# Determine the number of distinct folder rows removed
invalid_folder_count = invalid_df.select('folder').distinct().count()
print("%d distinct invalid folders found" % invalid_folder_count)

# Final analysis and delivery

- Analysis calculations are the process of using the columns of data in a DataFrame to compute some useful value using Spark's functionality.

- Spark UDFs are very powerful and flexible and are sometimes the only way to handle certain types of data. Unfortunately UDFs do come at a performance penalty compared to the built-in Spark functions, especially for certain operations. The solution is to perform calculations `inline` if possible.





In [ ]:
def getAvgSale(saleslist):
      totalsales = 0
      count = 0
      for sale in saleslist:
          totalsales += sale[2] + sale[3]
          count += 2
      return totalsales / count

udfGetAvgSale = udf(getAvgSale, DoubleType())
df = df.withColumn('avg_sale', udfGetAvgSale(df.sales_list))

# Inline Calculations
df = df.read.csv('datafile')
df = df.withColumn('avg', (df.total_sales / df.sales_count))
df = df.withColumn('sq_ft', df.width * df.length)
df = df.withColumn('total_avg_size', udfComputeTotal(df.entries) / df.numEntries)



In [ ]:
# Select the dog details and show 10 untruncated rows
print(joined_df.select('dog_list').show(10, truncate=False))

# Define a schema type for the details in the dog list
DogType = StructType([
	StructField("breed", StringType(), False),
    StructField("start_x", IntegerType(), False),
    StructField("start_y", IntegerType(), False),
    StructField("end_x", IntegerType(), False),
    StructField("end_y", IntegerType(), False)
])

In [ ]:
# Create a function to return the number and type of dogs as a tuple
def dogParse(doglist):
  dogs = []
  for dog in doglist:
    (breed, start_x, start_y, end_x, end_y) = dog.split(',')
    dogs.append((breed, int(start_x), int(start_y), int(end_x), int(end_y)))
  return dogs

# Create a UDF
udfDogParse = F.udf(dogParse, ArrayType(DogType))

# Use the UDF to list of dogs and drop the old column
joined_df = joined_df.withColumn('dogs', udfDogParse('dog_list')).drop('dog_list')

# Show the number of dogs in the first 10 rows
joined_df.select(F.size('dogs')).show(10)

In [ ]:
# Define a UDF to determine the number of pixels per image
def dogPixelCount(doglist):
  totalpixels = 0
  for dog in doglist:
    totalpixels += (dog[3] - dog[1]) * (dog[4] - dog[2])
  return totalpixels

# Define a UDF for the pixel count
udfDogPixelCount = F.udf(dogPixelCount, IntegerType())
joined_df = joined_df.withColumn('dog_pixels', udfDogPixelCount('dogs'))

# Create a column representing the percentage of pixels
joined_df = joined_df.withColumn('dog_percent', (joined_df.dog_pixels / (joined_df.width * joined_df.height)) * 100)

# Show the first 10 annotations with more than 60% dog
joined_df.where('dog_percent > 60').show(10)